In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk.classify.util 
from nltk.classify import NaiveBayesClassifier 
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import pandas as pd
from nltk.corpus import movie_reviews
from nltk.stem import PorterStemmer
import re
import numpy as np
import os

In [ ]:
###functions for regularzation and parsing

def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

def init_regularization(text, path):
    allsents = []
    current = os.getcwd()
    os.chdir(path)

    for filename in os.listdir(os.getcwd()):
        #print(filename)
        csv = os.path.join(os.getcwd(), str(filename))
        df = pd.read_csv(csv)
        allcomments = list(df[text])
        for comment in allcomments:
            ##remove \n
            comment = re.sub(r"\\n", "", str(comment))
            ##remove links
            comment = re.sub(r"http(\S)*", "", str(comment))
            ##remove deleted comments
            comment = re.sub(r"\[deleted\]", "", str(comment))
            ##remove emoji
            comment = remove_emojis(str(comment))
            ##remove extra space
            comment = comment.split()
            ' '.join(comment)       
            allsents.append(comment)
    os.chdir(current)
    return allsents

In [ ]:
##enter full directory
from google.colab import drive
#drive.mount('/content/drive/')

#!ls
sents = init_regularization('comment_body',"/content/drive/MyDrive/ColabNotebooks/subreddit-comments-dl-master/src/dataset/nominated_nonNominated_reddit_comments/")
print(len(sents), " number of sentences extracted")

766735  number of sentences extracted


In [ ]:
#nltk.download('punkt')
#nltk.download('vader_lexicon')
sia = SIA()
target_sentiments = []
neg_comments = []
pos_comments = []
words = []
pos_words = []
neg_words = []
neu_words = []
iter=0
## rate words based on coumpound sentiment threshhold
for sent in sents:
    iter += 1
    print(iter)
    string = ""
    for word in sent:
        string += word + " "
    string = string.lower()
    sent_pol_score = sia.polarity_scores(string)
    if sent_pol_score['compound'] >= 0.2:
        target_sentiments.append(1)
        pos_comments.append(sent)
        temp = word_tokenize(string)
        for i in temp:
            pos_words.append(i)

    elif(sent_pol_score['compound']) <= -0.2:
        target_sentiments.append(-1)
        neg_comments.append(sent)
        temp = word_tokenize(string)
        for i in temp:
            neg_words.append(i)
    else:
        target_sentiments.append(0)
        temp = word_tokenize(string)
        for i in temp:
            neu_words.append(i)

##tokenize words
unique_pos_words = set(pos_words)
unique_neg_words = set(neg_words)
unique_neu_words = set(neu_words)

print(len(unique_pos_words),"# of unique positive words")
print(len(unique_neg_words),"# of unique negative words")
print(len(unique_neu_words),"# of unique neutral words")

Streaming output truncated to the last 5000 lines.
761739
761740
761741
761742
761743
761744
761745
761746
761747
761748
761749
761750
761751
761752
761753
761754
761755
761756
761757
761758
761759
761760
761761
761762
761763
761764
761765
761766
761767
761768
761769
761770
761771
761772
761773
761774
761775
761776
761777
761778
761779
761780
761781
761782
761783
761784
761785
761786
761787
761788
761789
761790
761791
761792
761793
761794
761795
761796
761797
761798
761799
761800
761801
761802
761803
761804
761805
761806
761807
761808
761809
761810
761811
761812
761813
761814
761815
761816
761817
761818
761819
761820
761821
761822
761823
761824
761825
761826
761827
761828
761829
761830
761831
761832
761833
761834
761835
761836
761837
761838
761839
761840
761841
761842
761843
761844
761845
761846
761847
761848
761849
761850
761851
761852
761853
761854
761855
761856
761857
761858
761859
761860
761861
761862
761863
761864
761865
761866
761867
761868
761869
761870
761871
761872
761873
7618

In [ ]:
##This is how the Naive Bayes classifier expects the input
def create_word_features(words):
    useful_words = [word for word in words if word not in stopwords.words("english")]
    my_dict = dict([(word, True) for word in useful_words])
    return my_dict 

In [ ]:
## create a list of dictionary where each word is a key and pos/neg as value
## as term document matrix
nltk.download('stopwords')
neg_reviews = [] 
pos_reviews = []

for comment in neg_comments:
    neg_reviews.append((create_word_features(comment),"negative")) 

for comment in pos_comments:
    pos_reviews.append((create_word_features(comment),"positive"))
      
print(len(pos_reviews))
print(len(neg_reviews))

##splitting training set
train_set = []
test_set = []

outOften = 8

for i in range(0,int((len(neg_reviews)*outOften)/10)):
    train_set.append(neg_reviews[i])
for i in range(0,int((len(pos_reviews)*outOften)/10)):
    train_set.append(pos_reviews[i])
for i in range(int((len(neg_reviews)*outOften)/10),len(neg_reviews)):
    test_set.append(neg_reviews[i])
for i in range(int((len(pos_reviews)*outOften)/10), len(pos_reviews)):
    test_set.append(pos_reviews[i])

print("train dataset size:", len(train_set))
print("test dataset size:", len(test_set))
np.random.shuffle(test_set)
np.random.shuffle(train_set)

print("begin training")
classifier = NaiveBayesClassifier.train(train_set)

print("finish training")

accuracy = nltk.classify.util.accuracy(classifier, test_set)

print(accuracy)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


331398
212432
train dataset size: 435063
test dataset size: 108767
begin training
finish training
0.77102429964971
